# Problem 11.8

## Part a

In [1]:
import delimited ./data/chapter11/GerberGreenBook_Chapter11_Table_11_3, clear

(10 vars, 26 obs)


In [2]:
tabstat m1_x2 m2_x2, statistics(sum) by(region)


Summary statistics: sum
  by categories of: region (Region)

  region |     m1_x2     m2_x2
---------+--------------------
       1 |       .67      1.14
       2 |      2.17      2.53
       3 |       .45        .5
       4 |       7.1     13.36
       5 |      2.14      2.53
       6 |      1.91      2.05
---------+--------------------
   Total |     14.44     22.11
------------------------------


## Part b

In [3]:
clear 

import delim ./data/chapter11/Chapter_11_Dupas_2010_Dataset, clear
//import delim Dupas_QJE_2010, clear


gen log_price = log(price)

gen purchased =.
replace purchased = 1 if purchasednet=="yes"
replace purchased = 0 if purchasednet=="no"

rename cfw_id region	
	
drop if (price ==0 | region==4)


qui glm purchased log_price i.region c.log_price#i.region, family(binomial) link(logit)
predict double preds_1

qui glm purchased log_price i.region, family(binomial) link(logit)
predict double preds_2



generate non_p =.
replace non_p=1 if purchased==0
replace non_p=0 if purchased!=0

generate pred_nonp_1 = 1-preds_1
generate pred_nonp_2 = 1-preds_2



(3 vars, 1120 obs)

(117 missing values generated)

(1,120 missing values generated)

(509 real changes made)

(611 real changes made)


(301 observations deleted)


(option mu assumed; predicted mean purchased)


(option mu assumed; predicted mean purchased)

(819 missing values generated)

(473 real changes made)

(346 real changes made)




In [4]:
collapse (sum) purchases=purchased ///
   non_purchases=non_p ///
   pred_purchases_1=preds_1 ///
   pred_nonpurchases_1=pred_nonp_1 ///
   pred_purchases_2=preds_2 ///
   pred_nonpurchases_2=pred_nonp_2, by(region price)

In [5]:
gen chi_square_1 = (purchases- pred_purchases_1)^2/ pred_purchases_1 + ///
(non_purchases- pred_nonpurchases_1)^2/ pred_nonpurchases_1

gen chi_square_2 = (purchases- pred_purchases_2)^2/ pred_purchases_2 + ///
( non_purchases- pred_nonpurchases_2)^2/ pred_nonpurchases_2

In [6]:
// print table
list region price purchases non_purchases pred_purchases_1 ///
pred_nonpurchases_1 chi_square_1
list region price purchases non_purchases pred_purchases_2 ///
pred_nonpurchases_2 chi_square_2



     +------------------------------------------------------------------------+
     | region   price   purcha~s   non_pu~s   pred_pu~1   pred_n~1   chi_sq~1 |
     |------------------------------------------------------------------------|
  1. |      1      70         16         13   15.441598    13.5584   .0431908 |
  2. |      1     100         16         31   16.920757   30.07924    .078289 |
  3. |      1     130         12         37   12.291133   36.70887   .0092048 |
  4. |      1     190          5         23   3.8221953    24.1778    .420315 |
  5. |      1     250          2         28   2.5243177   27.47568   .1189098 |
     |------------------------------------------------------------------------|
  6. |      2      40         46         15   47.582604    13.4174   .2393082 |
  7. |      2      80         40         30   35.429156   34.57085   1.194044 |
  8. |      2     120         18         46   21.212681   42.78732   .7277873 |
  9. |      2     200         10      

In [7]:
qui tabstat chi_square_1, statistics(sum) save
scalar  model_1_chi_sq=el(r(StatTotal),1,1)

In [8]:
scalar pvalue_1=chiprob(9, model_1_chi_sq)
disp %8.5f model_1_chi_sq
disp %8.6f pvalue_1



 7.32282

0.603548


In [9]:
qui tabstat chi_square_2, statistics(sum) save
scalar  model_2_chi_sq=el(r(StatTotal),1,1)
scalar pvalue_2=chiprob(13, model_2_chi_sq)
disp %8.5f model_2_chi_sq
disp %8.6f pvalue_2





 7.68664

0.863399
